# Query Registry Users

Before executing this notebook, complete [step 2](./2_WorkflowRegistrySetup.ipynb).

In [1]:
lifemonitor_root = "/home/simleo/git/life_monitor"
%cd -q {lifemonitor_root}

In [2]:
import requests

In [3]:
lm_base_url = "https://localhost:8443"
lm_token_url = f"{lm_base_url}/oauth2/token"

In [4]:
# Get info on the "seek" registry
!docker-compose exec lm /bin/bash -c "flask registry show seek"



****************************************************************************************************
Workflow Registry 'seek' (uuid: 8ee6f93b-ac01-4183-b90e-14fec7ac9d22, type: seek_registry) registered!
****************************************************************************************************


OAuth2 settings to connect to LifeMonitor:
----------------------------------------------------------------------------------------------------
REGISTRY NAME: seek
REGISTRY API URL: https://seek:3000
REGISTRY CLIENT ID: fOeRRL3Z7tI1i5iHRczz4B0X
REGISTRY CLIENT SECRET: WA1rPDSudxDcQsZpqZosQXGwMx57hghbdMCaE7FBe9OmQLqE
REGISTRY CLIENT ALLOWED SCOPES: registry.info registry.user registry.workflow.read registry.workflow.write registry.user.workflow.read registry.user.workflow.write workflow.read workflow.write testingService.read testingService.write user.profile user.workflow.read
REGISTRY CLIENT ALLOWED FLOWS: ['client_credentials', 'authorization_code', 'refresh_token']
REGISTRY CLIEN

In [5]:
# Copy registry credentials from the above dump
CLIENT_ID = "fOeRRL3Z7tI1i5iHRczz4B0X"
CLIENT_SECRET = "WA1rPDSudxDcQsZpqZosQXGwMx57hghbdMCaE7FBe9OmQLqE"

In [6]:
# Enter the following URL in your browser
f"{lm_base_url}/oauth2/login/seek"
# Then log in as username: user1 and password: workflowhub

'https://localhost:8443/oauth2/login/seek'

In [7]:
# Get an authorization token from LifeMonitor
s = requests.session()
s.verify = False
s.headers.update({})
token_response = s.post(
    lm_token_url, 
    data={
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials",
        "scope": "registry.user"
    }, allow_redirects=True, verify=False)
assert token_response.status_code == 200
token = token_response.json()
token

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'access_token': 'xD84yAqT3tZ2rooVjDAwcAtXXLaW90ctxGRN1vgfyQ',
 'expires_in': 864000,
 'scope': 'registry.user',
 'token_type': 'Bearer'}

In [8]:
# Update headers with the OAuth2 token
s.headers.update({'Authorization': f"Bearer {token['access_token']}"})

In [9]:
# Get registry users
response = s.get(f"{lm_base_url}/registries/current/users")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_users = response.json()
registry_users

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'id': 2,
   'identities': {'seek': {'mbox_sha1sum': 'afbdc5cf6a75764623cd1a4abb114a15e1436f5e',
     'name': 'First User',
     'picture': None,
     'profile': '/people/2',
     'provider': {'name': 'seek',
      'type': 'registry',
      'uri': 'https://seek:3000',
      'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
     'sub': '2',
     'username': 'FirstUser'}},
   'username': 'FirstUser'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'resource': '/registries/current/users'}}